In [1]:
import tensorflow as tf
import input_data
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mnist = input_data.read_data_sets(".", one_hot=True)

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 1e-3
training_iters = 100000
batch_size = 128
display_step = 10

In [4]:
n_input = 784
n_classes = 10

In [5]:
dropout = 0.75

In [6]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [7]:
keep_prob = tf.placeholder(tf.float32)

In [8]:
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(\
        tf.nn.conv2d(img, w, strides=[1,1,1,1], padding='SAME'), b))

In [9]:
def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1],\
        strides=[1, k, k, 1], padding='SAME')

In [10]:
# 5x5 conv, 1 input, 32 outputs
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

In [11]:
# 5x5 conv, 32 inputs, 64 outputs
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

In [12]:
# FC, 7*7*64 inputs, 1024 outputs
wd1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))

In [13]:
# final pred layer, 1024 inputs, 10 predictions
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))

In [14]:
# Construct model
_X = tf.reshape(x, shape=[-1, 28, 28, 1])

In [15]:
# layer1
conv1 = max_pool( conv2d(_X, wc1, bc1), k=2 )
conv1 = tf.nn.dropout(conv1, keep_prob)

In [16]:
# layer2
conv2 = max_pool( conv2d(conv1, wc2, bc2), k=2)
conv2 = tf.nn.dropout(conv2, keep_prob)

In [17]:
# FC layer
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])
dense1 = tf.nn.relu( tf.matmul(dense1, wd1) + bd1)
dense1 = tf.nn.dropout(dense1, keep_prob)

In [18]:
# pred, cost
pred = tf.matmul(dense1, wout) + bout
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                        logits=pred, labels=y))

In [22]:
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(cost)

In [20]:
# Evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={\
          x: batch_xs, y: batch_ys, keep_prob:dropout })
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={\
                x: batch_xs, y: batch_ys, keep_prob:1.0})
            loss = sess.run(cost, feed_dict={\
                x: batch_xs, y: batch_ys, keep_prob:1.0})
            print ("Iter: ", str(step*batch_size),
                  ", Minibatch loss= ", "{:.6f}".format(loss),
                  ", Training Accuracy= ", "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")
    
    test_acc = sess.run(accuracy, feed_dict={\
                x: mnist.test.images[:256], \
                y: mnist.test.labels[:256], \
                keep_prob:1.0})
    print ("Testing Accuracy: ", test_acc)

Iter:  1280 , Minibatch loss=  44077.906250 , Training Accuracy=  0.07031
Iter:  2560 , Minibatch loss=  25155.757812 , Training Accuracy=  0.25781
Iter:  3840 , Minibatch loss=  15972.330078 , Training Accuracy=  0.48438
Iter:  5120 , Minibatch loss=  12187.164062 , Training Accuracy=  0.50781
Iter:  6400 , Minibatch loss=  8674.865234 , Training Accuracy=  0.66406
Iter:  7680 , Minibatch loss=  5425.223145 , Training Accuracy=  0.69531
Iter:  8960 , Minibatch loss=  6742.081055 , Training Accuracy=  0.67188
Iter:  10240 , Minibatch loss=  5503.110352 , Training Accuracy=  0.78125
Iter:  11520 , Minibatch loss=  4396.492188 , Training Accuracy=  0.79688
Iter:  12800 , Minibatch loss=  5319.227539 , Training Accuracy=  0.71875
Iter:  14080 , Minibatch loss=  4102.327637 , Training Accuracy=  0.76562
Iter:  15360 , Minibatch loss=  2946.980469 , Training Accuracy=  0.81250
Iter:  16640 , Minibatch loss=  2825.609375 , Training Accuracy=  0.85156
Iter:  17920 , Minibatch loss=  1820.2900

ValueError: Cannot feed value of shape (256, 784) for Tensor 'Placeholder_1:0', which has shape '(?, 10)'